In [14]:
# COS = "Courses-of-Study_2020-2021.pdf"
COS = "Courses-of-Study-2020-2021.pdf"

COURSE_DESCRIPTION_PAGE_FIRST = 161
COURSE_DESCRIPTION_PAGE_LAST = 370

PROGRAMME_PAGE_FIRST = 44
PROGRAMME_PAGE_LAST = 78

In [2]:
import fitz  # PyMuPDF
import re

In [3]:
COURSE_PATTERN = r"^[A-Z]{3}\d{3}"
CREDIT_PATTERN = r"^\d"
PREREQ_PATTERN = r"^Pre-requi"
OVERLAP_PATTERN = r"^Overlaps"

STANDARD_COS_PAGE_X1 = 49
STANDARD_COS_PAGE_X2 = 300
STANDARD_COS_PAGE_X3 = 596
STANDARD_COS_PAGE_Y1 = 84
STANDARD_COS_PAGE_Y2 = 790

In [4]:
def extract_text_by_coordinates(pdf_document, page_number, x1, y1, x2, y2):
    page = pdf_document[page_number - 1]
    rect = fitz.Rect(x1, y1, x2, y2)
    
    selected_text = page.get_text("text", clip=rect)
    return selected_text

def standard_cos_page_scraper(pdf_doc, page):
    x1, y1 = STANDARD_COS_PAGE_X1, STANDARD_COS_PAGE_Y1  # Starting coordinates of the rectangle
    x2, y2 = STANDARD_COS_PAGE_X2, STANDARD_COS_PAGE_Y2  # Ending coordinates of the rectangle
    x3, y3 = STANDARD_COS_PAGE_X2, STANDARD_COS_PAGE_Y1
    x4, y4 = STANDARD_COS_PAGE_X3, STANDARD_COS_PAGE_Y2
    left_side = extract_text_by_coordinates(pdf_doc, page, x1, y1, x2, y2)
    right_side = extract_text_by_coordinates(pdf_doc, page, x3, y3, x4, y4)
    return [left_side, right_side]


def scrape_courses_description():
    scraped_dataset = []
    
    first_page_i = COURSE_DESCRIPTION_PAGE_FIRST
    last_page_i = COURSE_DESCRIPTION_PAGE_LAST


    pdf_doc = fitz.open(COS)
    for page in range(first_page_i, last_page_i + 1):
        scraped_dataset.extend(standard_cos_page_scraper(pdf_doc, page))

    pdf_doc.close()
    
    return scraped_dataset

def attach_courses_description(scraped_dataset):
    for i in range(len(scraped_dataset)):
        data = scraped_dataset[i]
        if re.match(COURSE_PATTERN, data):
            scraped_dataset[i] = "\n" + data
        else :
            scraped_dataset[i] = " " + data
    return "".join(scraped_dataset)

def string_course_dataset_to_json(single_string_dataset):
    all_lines = single_string_dataset.split("\n")[1:]
    all_courses = []
    course = {}
    is_first_course = True
    for line in all_lines:
        if re.match(COURSE_PATTERN, line):
            if is_first_course: is_first_course = False
            else: all_courses.append(course)
            course = {
                "code" : line[:6],
                "name" : line[7:],
                "description" : "",
                "preReq" : "",
                "overLap" : "",
                "credits" : ""
            }
        elif re.match(CREDIT_PATTERN, line): course["credits"] = line
        elif re.match(PREREQ_PATTERN, line): course["preReq"] = line
        elif re.match(OVERLAP_PATTERN, line): course["overLap"] = line
        else: course["description"]  += line
    all_courses.append(course)

    return all_courses



def get_all_courses_description():
    scraped_dataset : list[str] = scrape_courses_description()
    single_string_dataset : str = attach_courses_description(scraped_dataset)
    courses_json_list : list[object] = string_course_dataset_to_json(single_string_dataset)
    return courses_json_list



# -------------------------------------------------------------------------------------------------------------
all_courses = get_all_courses_description()
# -------------------------------------------------------------------------------------------------------------


## TODO
modify the pre-req structure and credit structure



#### THE EDGE CASES
##### COS 20-21
- edge-case : page number 365
- prereq wrong in HUL275
- weird description in SBL724
- multiline pre-req in SBL720, SBL708, SBL710
- multiline course name in ASD882, and also there is a not allowed condition there in the name
- multiline course anem in CRL725
- Non-Credit Mandatory field in TXT800
- HUL modifications


In [7]:
for ele in all_courses:
    if ele["code"] == "ELL820":
        print(ele)

{'code': 'ELL820', 'name': 'Photonic Switching and Networking', 'description': 'Study of different types of networks, the enabling technologies and devices. Broadcast and Select network. Single and Multi-hop networks with example of Access networks, PoNS etc., Wavelength Routing network, virtual topology, Metro and Wide area networks. Wavelength Routing and Assignment, Traffic Grooming and Protection, Network Control and Management, optical packet and burst switching, Network Simulation Tools and Design guidelines.', 'preReq': '', 'overLap': '', 'credits': '3 Credits (3-0-0)'}


In [29]:
def scrape_course_header(pdf_doc, page_i):
    X1, X2 = 49, 596
    Y1, Y2 = 0, 84
    return extract_text_by_coordinates(pdf_doc, page_i, X1, Y1, X2, Y2).split("\n")

def is_course_table_page(pdf_doc, page_i):
    X1, X2 = 526, 596
    Y1, Y2 = 60, 160
    return "TOTAL" in extract_text_by_coordinates(pdf_doc, page_i, X1, Y1, X2, Y2)
    
def get_all_course_dependencies():
    FIRST_PAGE_I = PROGRAMME_PAGE_FIRST
    LAST_PAGE_I = PROGRAMME_PAGE_LAST

    pdf_doc = fitz.open(COS)
    all_courses = []
    curr_course = {
        "header" : "",
        "all_data" : []
    }
    prev_table_page = True
    for page_i in range(FIRST_PAGE_I, LAST_PAGE_I + 1):
        if is_course_table_page(pdf_doc, page_i):
            curr_course["all_data_joined"] = "\n".join(curr_course["all_data"])
            del curr_course["all_data"]
            all_courses.append(curr_course)
            curr_course = {
                "header" : "",
                "all_data" : []
            }
            prev_table_page = True
            continue
        if prev_table_page:
            prev_table_page = False
            curr_course["header"] = scrape_course_header(pdf_doc, page_i)
        curr_course["all_data"].extend( standard_cos_page_scraper(pdf_doc, page_i))
            

    pdf_doc.close()
    print(len(all_courses))
    for course in all_courses:
        print(course["header"])
    print("------------------------------------------------------------")
    print(all_courses[0]["all_data_joined"])
    
get_all_course_dependencies()

16
['Programme Code: AM1', 'Bachelor of Technology in Engineering and Computational Mechanics', 'Department of Applied Mechanics', '']
['Programme Code: BB1', 'Bachelor of Technology in Biochemical Engineering and Biotechnology', 'Department of Biochemical Engineering and Biotechnology', '']
['Programme Code: CH1', 'Bachelor of Technology in Chemical Engineering', 'Department of Chemical Engineering', '']
['Programme Code: CH7', 'Dual Degree Programme : Bachelor of Technology and Master of Technology ', 'in Chemical Engineering', '']
['Programme Code: CE1', 'Bachelor of Technology in Civil Engineering', 'Department of Civil Engineering', '']
['Programme Code: CS1', 'Bachelor of Technology in Computer Science and Engineering', 'Department of Computer Science and Engineering', '']
['Programme Code: CS5', 'Dual Degree Programme: Bachelor of Technology and Master of Technology ', 'in Computer Science and Engineering', '']
['Programme Code: EE1', 'Bachelor of Technology in Electrical Engine